In [3]:
import dns.name
import dns.query
import dns.dnssec
import dns.message
import dns.resolver
import dns.rdatatype
import re
import ipaddress
import time
import datetime
import random

In [331]:
rdtype_dic = {
    'A': 1      ,
    'NS':2      ,
    'DS':43     ,
    'RRSIG': 46 ,
    'DNSKEY':48 ,
}

In [4]:
def print_response(response):
    '''Print the response's answer, authority, and additional
    
    Args:
        response (dns.message.Message): a response of a single iterative DNS query
    '''
    print('\n', 'QUESTION:')
    for i in response.question:
        print(i.to_text())
    
    print('\n', 'ANSWER:')
    for i in response.answer:
        print(i.to_text())
    
    print('\n', 'AUTHORITY:')
    for i in response.authority:
        print(i.to_text())
        
    print('\n', 'ADDITIONAL:')
    for i in response.additional:
        print(i.to_text())

# Paypal

In [26]:
a_query = dns.message.make_query('paypal.com', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print_response(response1)


 QUESTION:
paypal.com. IN A

 ANSWER:

 AUTHORITY:
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 86400 IN DS 30909 8 2 e2d3c916f6deeac73294e8268fb5885044a833fc5459588f4a9184cfc41a5766
com. 86400 IN RRSIG DS 8 1 86400 20180301050000 20180216040000 41824 . I2raJjuftL0eqaPhDAjDsEY9Vk3FT/JL PQxSdBc4l9eHRvrcrQyG/k/YC8tieUxX YVWDkf0Ce7IpYwqBGhmitNpKbSppa80U ncojuMmNdWsq/JBk7eTyq1n1czYwCZRy 4iIEqsNkZICwBkXfkkUGe+gowXqpamuZ iCsHIU3Aw1KUtw8+n0bWbpYq+1worduP twhEyZXZd30+7kz3wMUQIABh4jOp3Ckm MDF6cfJssXrweaE+eTtddck86tj0YPgd iJOQcB3fv20BxXYs44HO17phtG0

In [401]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print_response(response2)


 QUESTION:
. IN DNSKEY

 ANSWER:
. 172800 IN DNSKEY 256 3 8 AwEAAaDJd0KOMYGCEF0/cftC2hrFtz5G Sn1HOiaxEp053AfbxQ3pT8BEtahPiUkC o1Qx4PECJ23YwaFhfWWjapr6AFxhD8kl fZGp95ickoRlm91ZzXX/mcfn9vlUpZK2 M8qjljNMzZJSopFY+cxRvib2Irb6YeP2 a0vppaLnvR4BeOyEkQolLqvVHW7UqDFi P/CM15BWBsAIdbyo8L1h3OeP63TaYIrW ttjGBILeZinSaJ39amiVs8t00RjTaKVo 3vY2k6dje1Rh1ELqjNj8+cKA8iWC3VU7 ApkyuGDy631RDILa6wCgcBVCzfFfOthQ ILxQra88tNWzCVoryQ89f1WjBJc=
. 172800 IN DNSKEY 257 3 8 AwEAAagAIKlVZrpC6Ia7gEzahOR+9W29 euxhJhVVLOyQbSEW0O8gcCjFFVQUTf6v 58fLjwBd0YI0EzrAcQqBGCzh/RStIoO8 g0NfnfL2MTJRkxoXbfDaUeVPQuYEhg37 NZWAJQ9VnMVDxP/VHL496M/QZxkjf5/E fucp2gaDX6RS6CXpoY68LsvPVjR0ZSwz z1apAzvN9dlzEheX7ICJBBtuA6G3LQpz W5hOA2hzCTMjJPJ8LbqF6dsV6DoBQzgu l0sGIcGOYl7OyQdXfZ57relSQageu+ip AdTTJ25AsRTAoub8ONGcLmqrAmRLKBP1 dfwhYB4N7knNnulqQxA+Uk1ihz0=
. 172800 IN DNSKEY 257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexT BAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq 7HrxRixHlFlExOLAJr5emLvN7SWXgnLh 4+B5xQlNVz8Og8kvArMtNROxVQuCaSnI DdD5LKyWbRd2n9WGe2R8PzgCmr3EgVLr jy

In [344]:
def get_rrset(response, rdtype):
    '''Get the desired rrset (DNSKEY, DS, A, NS), RRSIG and name from the response, their RRSIG
    
    Args:
        response (dns.message.Message): a response of a single iterative DNS query
        rdtype (str): rrset type
        
    Return:
        (rrset, rrsig, name) of desired rdtype
    '''
    try:
        if rdtype == 'DNSKEY' or rdtype == 'A':
            dnskey_or_a, rrsig, name = '', '', ''
            for rrset in response.answer:      # from observation, DNSKEY and A record is in ANSWER section
                if rrset.rdtype == rdtype_dic['RRSIG']:
                    rrsig = rrset
                else:   # rrset.rdtype == rdtype_dic['DNSKEY'] or ['A']:
                    dnskey_or_a = rrset
                    name = rrset.name
            return dnskey_or_a, rrsig, name
        if rdtype == 'DS' or rdtype == 'NS':
            ds_or_ns, rrsig, name = '', '', ''
            for rrset in response.authority:   # from observation, DS and NS record is in AUTHORITY section
                if rrset.rdtype == rdtype_dic['RRSIG']:
                    rrsig = rrset
                else:
                    ds_or_ns = rrset
                    name = rrset.name
            return ds_or_ns, rrsig, name
    except Exception as e:
        print('Oops! Bug in get_rrset')
        raise e          

In [388]:
def verify_dnskey(response):
    '''Verify the dnskey in response. If success, return name_key and dnskey
    
    Args:
        response (dns.message.Message): a response that contains DNSKEY record
        
    Return:
        return (dnskey, name) if success
    '''
    try:
        dnskey, rrsig_key, name_key = get_rrset(response, 'DNSKEY')
        dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
    except Exception as e:
        raise e
    else:
        print('Congrats! DNSKEYs are good~', name_key)
        return name_key, dnskey

In [389]:
verify_dnskey(response2)

Congrats! DNSKEYs are good~ .


(<DNS name .>, <DNS . IN DNSKEY RRset>)

In [390]:
# verify the DNSKEY rrset

try:
    dnskey, rrsig_key, name_key = get_rrset(response2, 'DNSKEY')
    dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DNSKEYs are good~')

Congrats! DNSKEYs are good~


<DNS name .>

In [357]:
type(dnskey)

dns.rrset.RRset

In [244]:
# verify the DS rrset

try:
    ds, rrsig_ds, name_ds = get_rrset(response1, 'DS')
    dns.dnssec.validate(ds, rrsig_ds, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DS is good~')

Congrats! DS is good~


In [368]:
trust_anchors = [
    # KSK-2017:
    dns.rrset.from_text('.', 1    , 'IN', 'DNSKEY', '257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexTBAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq7HrxRixHlFlExOLAJr5emLvN7SWXgnLh4+B5xQlNVz8Og8kvArMtNROxVQuCaSnIDdD5LKyWbRd2n9WGe2R8PzgCmr3EgVLrjyBxWezF0jLHwVN8efS3rCj/EWgvIWgb9tarpVUDK/b58Da+sqqls3eNbuv7pr+eoZG+SrDK6nWeL3c6H5Apxz7LjVc1uTIdsIXxuOLYA4/ilBmSVIzuDWfdRUfhHdY6+cn8HFRm+2hM8AnXGXws9555KrUB5qihylGa8subX2Nn6UwNR1AkUTV74bU='),
    # KSK-2010:
    dns.rrset.from_text('.', 15202, 'IN', 'DNSKEY', '257 3 8 AwEAAagAIKlVZrpC6Ia7gEzahOR+9W29euxhJhVVLOyQbSEW0O8gcCjF FVQUTf6v58fLjwBd0YI0EzrAcQqBGCzh/RStIoO8g0NfnfL2MTJRkxoX bfDaUeVPQuYEhg37NZWAJQ9VnMVDxP/VHL496M/QZxkjf5/Efucp2gaD X6RS6CXpoY68LsvPVjR0ZSwzz1apAzvN9dlzEheX7ICJBBtuA6G3LQpz W5hOA2hzCTMjJPJ8LbqF6dsV6DoBQzgul0sGIcGOYl7OyQdXfZ57relS Qageu+ipAdTTJ25AsRTAoub8ONGcLmqrAmRLKBP1dfwhYB4N7knNnulq QxA+Uk1ihz0='),
]

In [154]:
def get_anchor(year):
    ''' There are two anchors, get one of the anchors
    
    Args:
        year (int): 2017 or 2010
        
    Return:
        trusted root key singing key (str)
    '''
    if year == 2017:
        return trust_anchors[0].items[0].to_text()
    elif year == 2010:
        return trust_anchors[1].items[0].to_text()
    else:
        raise Exception('Parameter is neither 2017 nor 2010')

In [246]:
response2.answer[0]

<DNS . IN DNSKEY RRset>

In [361]:
# verify the root: compare the trust_anchors with the root's DNSKEY response

dnskeys, rrsig, name = get_rrset(response2, 'DNSKEY')
flag_trust = True
for dnskey in dnskeys:
    if dnskey.flags == 257:
        try:
            if dnskey.to_text() == get_anchor(2017):
                continue
            elif dnskey.to_text() == get_anchor(2010):
                continue
            else:
                flag_trust = False
                break
        except Exception as e:
            print(e)
            
print(flag_trust)

True


In [369]:
verify_root(dnskeys)

Root verified


In [366]:
def verify_root(dnskey):
    '''Verify the root by comparing the pubksk in the response and the trusted pubksk
    
    Args:
        dnskey (dns.rrset.RRset)
        
    
    '''
    for dnskey in dnskeys:
        if dnskey.flags == 257:
            if dnskey.to_text() == get_anchor(2017):
                continue
            elif dnskey.to_text() == get_anchor(2010):
                continue
            else:
                raise Exception('Does not match trusted pubksk')
    else:
        print('Root verified')

In [359]:
type(dnskeys)

dns.rrset.RRset

In [165]:
response3 = dns.query.udp(a_query, '192.5.6.30')
print_response(response3)


 QUESTION:
paypal.com. IN A

 ANSWER:

 AUTHORITY:
paypal.com. 172800 IN NS pdns100.ultradns.net.
paypal.com. 172800 IN NS pdns100.ultradns.com.
paypal.com. 172800 IN NS ns1.p57.dynect.net.
paypal.com. 172800 IN NS ns2.p57.dynect.net.
paypal.com. 86400 IN DS 21037 5 2 0df17b28554954d819e0ceeab98fcfcd56572a4cf4f551f0a9be6d04db2f65c3
paypal.com. 86400 IN RRSIG DS 8 2 86400 20180221051534 20180214040534 46967 com. x/kxI5vzp8cb6XXEKKyGBWz/2ERUzqgs 9W1w+axifzpM3yViJEQIKTbKL2l+1qY+ Myh+F4EndVY9Bq61VLtLpfLMFfD8/c/B s9/kRAnxrGDZL7KBRiikTrMIfbJHFNKw hpVX8vAFuFm/HIr9vYmiohLf59BkTh6B jJ3+vBC+Quw=

 ADDITIONAL:
pdns100.ultradns.net. 172800 IN A 156.154.65.100
pdns100.ultradns.net. 172800 IN AAAA 2610:a1:1014::88
pdns100.ultradns.com. 172800 IN A 156.154.64.100
pdns100.ultradns.com. 172800 IN AAAA 2001:502:f3ff::88
ns1.p57.dynect.net. 172800 IN A 208.78.70.57
ns2.p57.dynect.net. 172800 IN A 204.13.250.57


In [166]:
c_query = dns.message.make_query('com.', 'DNSKEY', want_dnssec=True)
response4 = dns.query.udp(c_query, '192.5.6.30')
print_response(response4)


 QUESTION:
com. IN DNSKEY

 ANSWER:
com. 86400 IN DNSKEY 256 3 8 AQPLmJ3YIDDZPh9wYbc76pe1azHsCyIi DzBgf7fz+k4poMWHzE6f4My3VcPX2SIn /v0jPGBIEVZl2pPdY/IqOd5BfCsjrE+b BUYz6uxWiaT3wMuhfN1LNOazLQRlgU8R rEOgG1L4yFD8m9FVAisWsXLY+WQ8qIyr K/UIUwDmiFC5vQ==
com. 86400 IN DNSKEY 257 3 8 AQPDzldNmMvZFX4NcNJ0uEnKDg7tmv/F 3MyQR0lpBmVcNcsIszxNFxsBfKNW9JYC Yqpik8366LE7VbIcNRzfp2h9OO8HRl+H +E08zauK8k7evWEmu/6od+2boggPoiEf GNyvNPaSI7FOIroDsnw/taggzHRX1Z7S OiOiPWPNIwSUyWOZ79VmcQ1GLkC6NlYv G3HwYmynQv6oFwGv/KELSw7ZSdrbTQ0H XvZbqMUI7BaMskmvgm1G7oKZ1YiF7O9i oVNc0+7ASbqmZN7Z98EGU/Qh2K/BgUe8 Hs0XVcdPKrtyYnoQHd2ynKPcMMlTEih2 /2HDHjRPJ2aywIpKNnv4oPo/
com. 86400 IN RRSIG DNSKEY 8 1 86400 20180303192533 20180216192033 30909 com. KfTfHhzBbmAT4lLfrR8eSGeXr4QY068j 7gDmr7kW4vmSVoCPxBblazcCtT5Ew2um t0ZmyzkLzgxjjvtt7jgkML3z59HZRHm7 Y4y++xPcOQmK8CkeHhJnkoRdLeKxEZwS tlVBgML/udxBRlTp7c4YL7i62zfDNapa TqLWuXxsgOadOlcNgYRQH9qRWWc5578/ z7oP4NcHugOPCOgMPhXAJwn2Be5I9BZN SjeeJFiaG2xVuxuUqrArEdF6UZi2TB8+ U5Dcu3EGENIucovnt1O574azHc

In [250]:
# verify the DNSKEY rrset

try:
    dnskey, rrsig_key, name_key = get_rrset(response4, 'DNSKEY')
    dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DNSKEYs are good~')

Congrats! DNSKEYs are good~


In [251]:
# verify the DS rrset

try:
    ds, rrsig_ds, name_ds = get_rrset(response3, 'DS')
    dns.dnssec.validate(ds, rrsig_ds, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DS is good~')

Congrats! DS is good~


In [272]:
# verify the com. zone: do a hash on com's public key signing key, then see if equals to the DS in parent

rrset = response1.authority[1]
# rrset.rdtype == 43
trust_ds = rrset.items[0]
algorithm = 'SHA256' if trust_ds.digest_type == 2 else 'SHA1' 
name = dns.name.from_text('com.')
com_pubksk = response4.answer[0].items[1]
ds = dns.dnssec.make_ds(name, com_pubksk, algorithm)

if ds == trust_ds:
    print(True)
else:
    print(False)

True


In [323]:
# verify the com. zone: do a hash on com's public key signing key, then see if equals to the DS in parent

trust_ds, name = get_trust_ds(response1)
algorithm = 'SHA256' if trust_ds.digest_type == 2 else 'SHA1' 
pubksk = get_pubksk(response4)
ds = dns.dnssec.make_ds(name, pubksk, algorithm)

if ds == trust_ds:
    print(True)
else:
    print(False)

True


In [412]:
def verify_ds(response, name_key, dnskey):
    '''Verify the ds in the response.
    
    Args:
        response (dns.message.Message): a response that contains DS record
        name_key (dns.name.Name): name of zone that contains the DNSKEY
        dnskey (dns.rrset.RRset): rrset that contains public zone signing key
    '''
    try:
        ds, rrsig_ds, name_ds = get_rrset(response, 'DS')
        dns.dnssec.validate(ds, rrsig_ds, {name_key:dnskey})
    except Exception as e:
        raise e
    else:
        print('Congrats! DS is good~', name_ds)

In [396]:
def verify_zone(response, response_parent):
    '''Verify the zone: do a hash on the zone's public key signing key, then see if equals to the DS in parent
    
    Args:
        response (dns.message.Message): a response that contains pubksk to verify
        response_parent (dns.message.Message): a parent response that has the trusted DS
    '''
    trust_ds, name = get_trust_ds(response_parent)
    algorithm = 'SHA256' if trust_ds.digest_type ==2 else 'SHA1'
    pubksk = get_pubksk(response)
    ds = dns.dnssec.make_ds(name, pubksk, algorithm)
    
    if ds != trust_ds:
        raise Exception('DS does not match!', name)
        print('caitao')
    print('zhan')

In [397]:
verify_zone(response4, response1)

zhan


In [320]:
def get_pubksk(response):
    '''Get public key signing key from response
    
    Args:
        response (dns.message.Message) that contains DNSKEY information
        
    Return:
        a (dns.rdtypes.ANY.DNSKEY.DNSKEY) that contains public key signing key
    '''
    dnskey, rrsig_key, name_key = get_rrset(response, 'DNSKEY')
    for item in dnskey:
        if item.flags == 257:
            return item

In [301]:
def get_trust_ds(response):
    '''Get trust ds digest from parent's response
    
    Args:
        response (dns.message.Message) from parent that contains child's DS information
        
    Return:
        trusted ds digest from parent (dns.rdtypes.ANY.DS.DS), and its name (dns.name.Name)
    '''
    ds, rrsig_ds, name_ds = get_rrset(response, 'DS')
    return ds.items[0], name_ds

In [324]:
response5 = dns.query.udp(a_query, '156.154.65.100')
print_response(response5)


 QUESTION:
paypal.com. IN A

 ANSWER:
paypal.com. 300 IN A 64.4.250.33
paypal.com. 300 IN A 64.4.250.32
paypal.com. 300 IN RRSIG A 5 2 300 20180227181123 20180128174703 11811 paypal.com. zJ22XRBBNn7dQJRrmpuCxuVqe8M8iCiD oQhN8Yi2969m+HbzJnc/3vz4/s6/TCYB w01uS0vclejIXj3VxLd0a06LR183J8GZ 7fneqNOVxuxxaRLSQ9GR4/sIQb4SMOrf UOfSfefCI+B4GV/7FHKGRDZr5LGnBCV5 At4mS4OmsfY=

 AUTHORITY:
paypal.com. 300 IN NS ns2.p57.dynect.net.
paypal.com. 300 IN NS pdns100.ultradns.net.
paypal.com. 300 IN NS pdns100.ultradns.com.
paypal.com. 300 IN NS ns1.p57.dynect.net.
paypal.com. 300 IN RRSIG NS 5 2 300 20180309013118 20180207003908 11811 paypal.com. IRK1zCNEdt8omQ8sToyldtkGiScPN58J geN49VpC2uMVe9GvhrC95cM1y0u7XQ07 3ZE+zZupGtux+/2jNs+JJSu9qy3JAQsT B52O8kwSwEt0LUbRqV9qhKj3qJdVbk1J fWaOI9PGaJA/kYcEHl03DzHbh8IaBRih h0r2KV3XWZE=

 ADDITIONAL:


In [326]:
d_query = dns.message.make_query('paypal.com.', 'DNSKEY', want_dnssec=True)
response6 = dns.query.udp(d_query, '156.154.65.100')
print_response(response6)


 QUESTION:
paypal.com. IN DNSKEY

 ANSWER:
paypal.com. 600 IN DNSKEY 256 3 5 AwEAAc/7r7w6qEg59vzyfcKgIm7K3h43 tglKOJjoFyK5lxhl2e7vh8Cwvj3cwKQH ccsvWuvAK0ummTSysxZT0JJgw8gkhIGG iZ63MJTcbocDezgZiA/q4ejpjdrj27gs 5mCHAsyC9BAVZiysfIwqtRFsP0GjivNY zIc6qGWAXcAJKLLX
paypal.com. 600 IN DNSKEY 257 3 5 AwEAAdVtmC6yOQb0+5MT5ezC9GJsCb10 WkVE1qMAbilAN5KZ0/wJD+4P1/WB7ctX C0RnEjHzVrKmLsFzRb3mpI9wm0cf5pN8 BHMSVfdDpycjNyGbMy7EKJ77POD7lSim JouMx5Tp+HQaJZeU0MnXJkR4qaAvShr5 iNtVaopOuIXQfRDnLdDoxETw9XplIL9w kpe1gW5uQNk+Fhy4PRyf+e5yhgiZjemm RiEDJH+FxCUAf8ZV+xHSuucWKY3V1tEh eptdUaIAbnCGDWGTbi9ziai6SrLzRrrD eImwm42PkxD5cURMWqHQIBfEJlDB+koq WPm73sDPMlPgoBUyxb/w5JFLV+c=
paypal.com. 600 IN RRSIG DNSKEY 5 2 600 20180308213939 20180206210605 11811 paypal.com. foJEGOUbST8u8Y7xbsg0f0xS5EMLNjnw GD+9/eH/VHyaGIDfEXIDbn+5Z5JkUoTv hOrp2aWtF3p13p2kkxXpSBO82lpaEpcF QwawMyEXhhAwzAW91I2hXDIiTKDQvTP5 eN6HTn9hpFLt6iy8GlmKX7j39Rqj8nWx c9w2r0igAbU=
paypal.com. 600 IN RRSIG DNSKEY 5 2 600 20180308213939 20180206210605 21037 paypa

In [327]:
# verify the DNSKEY rrset

try:
    dnskey, rrsig_key, name_key = get_rrset(response6, 'DNSKEY')
    dns.dnssec.validate(dnskey, rrsig_key, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! DNSKEYs are good~')

Congrats! DNSKEYs are good~


In [335]:
# verify the A rrset

try:
    a, rrsig_a, name_a = get_rrset(response5, 'A')
    dns.dnssec.validate(a, rrsig_a, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! A records are good~')

Congrats! A records are good~


In [348]:
# verify the NS rrset

try:
    ns, rrsig_ns, name_ns = get_rrset(response5, 'NS')
    dns.dnssec.validate(ns, rrsig_ns, {name_key:dnskey})
except Exception as e:
    print('Oops! Validation failure:', e)
else:
    print('Congrats! NS records are good~')

Congrats! NS records are good~


In [385]:
# verify the paypal.com. zone: do a hash on com's public key signing key, then see if equals to the DS in parent

trust_ds, name = get_trust_ds(response3)
algorithm = 'SHA256' if trust_ds.digest_type == 2 else 'SHA1' 
pubksk = get_pubksk(response6)
ds = dns.dnssec.make_ds(name, pubksk, algorithm)

if ds == trust_ds:
    print(True)
else:
    print(False)

True


# CNN

In [399]:
a_query = dns.message.make_query('cnn.com', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print_response(response1)


 QUESTION:
cnn.com. IN A

 ANSWER:

 AUTHORITY:
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.
com. 86400 IN DS 30909 8 2 e2d3c916f6deeac73294e8268fb5885044a833fc5459588f4a9184cfc41a5766
com. 86400 IN RRSIG DS 8 1 86400 20180303170000 20180218160000 41824 . dcLKco4JPBMDIFv6FgltBvt61/cM63Hq beeByi52saHDl3ZuF2ewjMYgWggSX5dq qoek8nmisG16xC51k64z4SOKwv3a6tqj DiQd3HJHUdcJnJauJTEmTQ2LMpVe3cxd eDkLZ7mJBR7txQJzbtgg9AwoYKjtLLHs 3aCpDizwrdu3pFS1YCAq8E4QKJ15rP1A eXLD3NvGT4R7PCkGxHE/P8TryiZPjw4c iCN9R1UOqJzLX2Jqr6wNvUr884X/l5qu gT4wphOEGdhCa6jTj0KZuTpxF9m/YM

In [403]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print_response(response2)


 QUESTION:
. IN DNSKEY

 ANSWER:
. 172800 IN DNSKEY 256 3 8 AwEAAaDJd0KOMYGCEF0/cftC2hrFtz5G Sn1HOiaxEp053AfbxQ3pT8BEtahPiUkC o1Qx4PECJ23YwaFhfWWjapr6AFxhD8kl fZGp95ickoRlm91ZzXX/mcfn9vlUpZK2 M8qjljNMzZJSopFY+cxRvib2Irb6YeP2 a0vppaLnvR4BeOyEkQolLqvVHW7UqDFi P/CM15BWBsAIdbyo8L1h3OeP63TaYIrW ttjGBILeZinSaJ39amiVs8t00RjTaKVo 3vY2k6dje1Rh1ELqjNj8+cKA8iWC3VU7 ApkyuGDy631RDILa6wCgcBVCzfFfOthQ ILxQra88tNWzCVoryQ89f1WjBJc=
. 172800 IN DNSKEY 257 3 8 AwEAAagAIKlVZrpC6Ia7gEzahOR+9W29 euxhJhVVLOyQbSEW0O8gcCjFFVQUTf6v 58fLjwBd0YI0EzrAcQqBGCzh/RStIoO8 g0NfnfL2MTJRkxoXbfDaUeVPQuYEhg37 NZWAJQ9VnMVDxP/VHL496M/QZxkjf5/E fucp2gaDX6RS6CXpoY68LsvPVjR0ZSwz z1apAzvN9dlzEheX7ICJBBtuA6G3LQpz W5hOA2hzCTMjJPJ8LbqF6dsV6DoBQzgu l0sGIcGOYl7OyQdXfZ57relSQageu+ip AdTTJ25AsRTAoub8ONGcLmqrAmRLKBP1 dfwhYB4N7knNnulqQxA+Uk1ihz0=
. 172800 IN DNSKEY 257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexT BAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq 7HrxRixHlFlExOLAJr5emLvN7SWXgnLh 4+B5xQlNVz8Og8kvArMtNROxVQuCaSnI DdD5LKyWbRd2n9WGe2R8PzgCmr3EgVLr jy

In [404]:
response3 = dns.query.udp(a_query, '192.5.6.30')
print_response(response3)


 QUESTION:
cnn.com. IN A

 ANSWER:

 AUTHORITY:
cnn.com. 172800 IN NS ns-47.awsdns-05.com.
cnn.com. 172800 IN NS ns-576.awsdns-08.net.
cnn.com. 172800 IN NS ns-1630.awsdns-11.co.uk.
cnn.com. 172800 IN NS ns-1086.awsdns-07.org.
CK0POJMG874LJREF7EFN8430QVIT8BSM.com. 86400 IN NSEC3 1 1 0 - ck0q1gin43n1arrc9osm6qpqr81h5m9a NS SOA RRSIG DNSKEY NSEC3PARAM
CK0POJMG874LJREF7EFN8430QVIT8BSM.com. 86400 IN RRSIG NSEC3 8 2 86400 20180225054804 20180218043804 46967 com. vvxe628HQa7sONZv2L/ptP74rr2WW1Hx mxvlM3WTPHCmsTN+UIBOo90Z1nksPuV5 FYcve0LrAMjA/ymC+53qe6evfYq8tSzz tmDju08vB9aHdEblHcLhye4Uto6aolUS 8vKHuavugWlXEzC/sblUdn1LXXfymspW 8vEzOdx2PV4=
FVT71LMDJ71M5N4BBJG7S42QT4H2K0VS.com. 86400 IN NSEC3 1 1 0 - fvt8070rvmmn14h33tu31073gpdt89uq NS DS RRSIG
FVT71LMDJ71M5N4BBJG7S42QT4H2K0VS.com. 86400 IN RRSIG NSEC3 8 2 86400 20180225055204 20180218044204 46967 com. AI3R9chO8Bv9JSpkQTlcP8eHJwKxf66E CVTBC1jJ7NsdpaJSxFigeQO2JC8MXA1W K2K25Sd0exSnHxM5doLBURnZEoBG3Qzk yBYMw8DX9j1FgCfuD3EqwHE3TwGlY7/Y U0zrCjPMUYT

In [406]:
def check_ds_exist(response):
    '''Check whether DS record exist in the response
    
    Args:
        response (dns.message.Message): a response
        
    Return:
        (bool) True or False
    '''
    flag = False
    for rrset in response.authority:
        if rrset.rdtype == rdtype_dic['DS']:
            flag = True
            break
    return flag

In [409]:
check_ds_exist(response2)

False

In [405]:
c_query = dns.message.make_query('com.', 'DNSKEY', want_dnssec=True)
response4 = dns.query.udp(c_query, '192.5.6.30')
print_response(response4)


 QUESTION:
com. IN DNSKEY

 ANSWER:
com. 86400 IN DNSKEY 256 3 8 AQPLmJ3YIDDZPh9wYbc76pe1azHsCyIi DzBgf7fz+k4poMWHzE6f4My3VcPX2SIn /v0jPGBIEVZl2pPdY/IqOd5BfCsjrE+b BUYz6uxWiaT3wMuhfN1LNOazLQRlgU8R rEOgG1L4yFD8m9FVAisWsXLY+WQ8qIyr K/UIUwDmiFC5vQ==
com. 86400 IN DNSKEY 257 3 8 AQPDzldNmMvZFX4NcNJ0uEnKDg7tmv/F 3MyQR0lpBmVcNcsIszxNFxsBfKNW9JYC Yqpik8366LE7VbIcNRzfp2h9OO8HRl+H +E08zauK8k7evWEmu/6od+2boggPoiEf GNyvNPaSI7FOIroDsnw/taggzHRX1Z7S OiOiPWPNIwSUyWOZ79VmcQ1GLkC6NlYv G3HwYmynQv6oFwGv/KELSw7ZSdrbTQ0H XvZbqMUI7BaMskmvgm1G7oKZ1YiF7O9i oVNc0+7ASbqmZN7Z98EGU/Qh2K/BgUe8 Hs0XVcdPKrtyYnoQHd2ynKPcMMlTEih2 /2HDHjRPJ2aywIpKNnv4oPo/
com. 86400 IN RRSIG DNSKEY 8 1 86400 20180303192533 20180216192033 30909 com. KfTfHhzBbmAT4lLfrR8eSGeXr4QY068j 7gDmr7kW4vmSVoCPxBblazcCtT5Ew2um t0ZmyzkLzgxjjvtt7jgkML3z59HZRHm7 Y4y++xPcOQmK8CkeHhJnkoRdLeKxEZwS tlVBgML/udxBRlTp7c4YL7i62zfDNapa TqLWuXxsgOadOlcNgYRQH9qRWWc5578/ z7oP4NcHugOPCOgMPhXAJwn2Be5I9BZN SjeeJFiaG2xVuxuUqrArEdF6UZi2TB8+ U5Dcu3EGENIucovnt1O574azHc

In [69]:
response6 = dns.query.udp(b_query, '205.251.192.47')
print_response(response6)


 QUESTION:
cnn.com. IN A

 ANSWER:
cnn.com. 60 IN A 151.101.1.67
cnn.com. 60 IN A 151.101.129.67
cnn.com. 60 IN A 151.101.193.67
cnn.com. 60 IN A 151.101.65.67

 AUTHORITY:
cnn.com. 3600 IN NS ns-1086.awsdns-07.org.
cnn.com. 3600 IN NS ns-1630.awsdns-11.co.uk.
cnn.com. 3600 IN NS ns-47.awsdns-05.com.
cnn.com. 3600 IN NS ns-576.awsdns-08.net.

 ADDITIONAL:


# dnssec-failed.org

In [808]:
a_query = dns.message.make_query('dnssec-failed.org', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print_response(response1)


 QUESTION:
dnssec-failed.org. IN A

 ANSWER:

 AUTHORITY:
org. 172800 IN NS a0.org.afilias-nst.info.
org. 172800 IN NS a2.org.afilias-nst.info.
org. 172800 IN NS b0.org.afilias-nst.org.
org. 172800 IN NS b2.org.afilias-nst.org.
org. 172800 IN NS c0.org.afilias-nst.info.
org. 172800 IN NS d0.org.afilias-nst.org.
org. 86400 IN DS 9795 7 1 364dfab3daf254cab477b5675b10766ddaa24982
org. 86400 IN DS 9795 7 2 3922b31b6f3a4ea92b19eb7b52120f031fd8e05ff0b03bafcf9f891bfe7ff8e5
org. 86400 IN RRSIG DS 8 1 86400 20180303170000 20180218160000 41824 . fwa48E6QXqBEVEKq3He6d7Uh8omalRyw yY4jQns8V2n6LFH5FHlEV1HJFWpKsIjs XbsLGOV4Yl8CrreTyHLHzD2IQCcvP331 pFUiF5UNNJr95z9iI1gMGN+usJIEJcuH 2936rlEsgAXjTD7ZqYiPVPm/VnyZrg2w xv3MQleUccRj5GMRtlaUqY5YmsacZ3C2 dm4CSdC+FodQgV8OIpqDrQa8PMuCddq2 GuVg3uTbi9xISvrhaGqxf2F7pFvbNwfc XeLK8ZoYHDLpkjbT2BGQq0qOj+rsaI8e Eb/52JQgKHkOvaDvTtFclk7oTlNhi0Yh xZcTYzntKrKupie3wd/Ffw==

 ADDITIONAL:
a0.org.afilias-nst.info. 172800 IN A 199.19.56.1
a2.org.afilias-nst.info. 172800 IN A 19

In [809]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print_response(response2)


 QUESTION:
. IN DNSKEY

 ANSWER:
. 172800 IN DNSKEY 256 3 8 AwEAAaDJd0KOMYGCEF0/cftC2hrFtz5G Sn1HOiaxEp053AfbxQ3pT8BEtahPiUkC o1Qx4PECJ23YwaFhfWWjapr6AFxhD8kl fZGp95ickoRlm91ZzXX/mcfn9vlUpZK2 M8qjljNMzZJSopFY+cxRvib2Irb6YeP2 a0vppaLnvR4BeOyEkQolLqvVHW7UqDFi P/CM15BWBsAIdbyo8L1h3OeP63TaYIrW ttjGBILeZinSaJ39amiVs8t00RjTaKVo 3vY2k6dje1Rh1ELqjNj8+cKA8iWC3VU7 ApkyuGDy631RDILa6wCgcBVCzfFfOthQ ILxQra88tNWzCVoryQ89f1WjBJc=
. 172800 IN DNSKEY 257 3 8 AwEAAagAIKlVZrpC6Ia7gEzahOR+9W29 euxhJhVVLOyQbSEW0O8gcCjFFVQUTf6v 58fLjwBd0YI0EzrAcQqBGCzh/RStIoO8 g0NfnfL2MTJRkxoXbfDaUeVPQuYEhg37 NZWAJQ9VnMVDxP/VHL496M/QZxkjf5/E fucp2gaDX6RS6CXpoY68LsvPVjR0ZSwz z1apAzvN9dlzEheX7ICJBBtuA6G3LQpz W5hOA2hzCTMjJPJ8LbqF6dsV6DoBQzgu l0sGIcGOYl7OyQdXfZ57relSQageu+ip AdTTJ25AsRTAoub8ONGcLmqrAmRLKBP1 dfwhYB4N7knNnulqQxA+Uk1ihz0=
. 172800 IN DNSKEY 257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexT BAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq 7HrxRixHlFlExOLAJr5emLvN7SWXgnLh 4+B5xQlNVz8Og8kvArMtNROxVQuCaSnI DdD5LKyWbRd2n9WGe2R8PzgCmr3EgVLr jy

In [810]:
name_key, dnskey = verify_dnskey(response2)
verify_ds(response1, name_key, dnskey)
verify_root(dnskey)

Congrats! DNSKEYs are good~ .
Congrats! DS is good~ org.
Root verified


In [811]:
response3 = dns.query.udp(a_query, '199.19.56.1')
print_response(response3)


 QUESTION:
dnssec-failed.org. IN A

 ANSWER:

 AUTHORITY:
dnssec-failed.org. 86400 IN NS dns105.comcast.net.
dnssec-failed.org. 86400 IN NS dns103.comcast.net.
dnssec-failed.org. 86400 IN NS dns102.comcast.net.
dnssec-failed.org. 86400 IN NS dns101.comcast.net.
dnssec-failed.org. 86400 IN NS dns104.comcast.net.
dnssec-failed.org. 86400 IN DS 106 5 1 4f219dce274f820ea81ea1150638dabe21eb27fc
dnssec-failed.org. 86400 IN DS 106 5 2 ae3424c9b171af3b202203767e5703426130d76ef6847175f2eed355f86ef1ce
dnssec-failed.org. 86400 IN RRSIG DS 7 2 86400 20180310152953 20180217142953 1862 org. Z2Ts/qpDhYz+yIknIHuUlv1R+no2Kocw 4kFw/genOTPvlXYvtW3fncYy2RgxthRO hCr36WH1qS5L+SmGNj2Q2c/6PO6G7VEy ouWSE3jNPUu31Khzt1gYtd9bqKWpp2CP Ajhrxj/qUdCkpM6OqM489o4t9zhrWMtZ c5rYdf02pvY=

 ADDITIONAL:


In [814]:
c_query = dns.message.make_query('org.', 'DNSKEY', want_dnssec=True)
response4 = dns.query.udp(c_query, '199.19.56.1')
print_response(response4)


 QUESTION:
org. IN DNSKEY

 ANSWER:

 AUTHORITY:

 ADDITIONAL:


In [701]:
def verify_org_dnskey(ip):
    '''Zone org. has to query dnskey and rrsig separately. 
       There are 4 DNSKEYs for org, but you can not get all 4 of them in a single query.
       You have to query multiple times and union them until you get all the four. Shit!!!
    
    Args:
        ip (str): one ip of an org name server
        
    Return:
        (dnskey, name) if success
    '''
    dnskey_org = None
    name_org = None
    query_org = dns.message.make_query('org.', 'DNSKEY')
    response_org = dns.query.udp(query_org, ip)
    while len(response_org.answer) == 0:
        response_org = dns.query.udp(query_org, ip)
    dnskey_org = response_org.answer[0]
    name_org = dnskey_org.name
    while len(dnskey_org) != 4:
        query_org = dns.message.make_query('org.', 'DNSKEY')
        response_org = dns.query.udp(query_org, ip)
        try:
            dnskey_org.union_update(response_org.answer[0])
        except:
            pass
    
    rrsig_dnskey_org = None
    query_org = dns.message.make_query('org.', 'RRSIG')
    response_org = dns.query.udp(query_org, ip)
    while len(response_org.answer) == 0:
        response_org = dns.query.udp(query_org, ip)
    rrsig_dnskey_org = response_org.answer[0]
    
    try:
        dns.dnssec.validate(dnskey_org, rrsig_dnskey_org, {name_org:dnskey_org})
    except Exception as e:
        raise e
    else:
        print('Congrats! DNSKEYs are good~', name)
        return name_org, dnskey_org

In [805]:
def verify_org_zone(dnskey_org, response_parent):
    '''Verify the zone: do a hash on the zone's public key signing key, then see if equals to the DS in parent
    
    Args:
        response (dns.message.Message): a response that contains pubksk to verify
        response_parent (dns.message.Message): a parent response that has the trusted DS
    '''
    trust_ds, name = get_trust_ds(response_parent)
    algorithm = 'SHA256' if trust_ds.digest_type ==2 else 'SHA1'
    
    dnskey_backup = dnskey_org
    
    for dnskey in dnskey_backup:
        if dnskey.flags == 256:
            dnskey_org.remove(dnskey)   # remove the public zone signing keys, so the public key signing keys are left
    
    ds2 = dns.dnssec.make_ds(name, dnskey_org.items[0], algorithm)
    ds1 = dns.dnssec.make_ds(name, dnskey_org.items[1], algorithm)
    if (ds1 == trust_ds or ds2 == trust_ds) == 0:
        raise Exception('DS does not match!', name)
    print('Congrads! Zone', name, 'verified')

In [815]:
name_key, dnskey = verify_org_dnskey('199.19.56.1')
verify_ds(response3, name_key, dnskey)
verify_org_zone(dnskey, response1)

Congrats! DNSKEYs are good~ org.
Congrats! DS is good~ dnssec-failed.org.
Congrads! Zone org. verified


In [816]:
response5 = dns.query.udp(a_query, '68.87.72.244')
print_response(response5)


 QUESTION:
dnssec-failed.org. IN A

 ANSWER:
dnssec-failed.org. 7200 IN A 69.252.80.75
dnssec-failed.org. 7200 IN RRSIG A 5 2 7200 20180220173528 20180213143028 44973 dnssec-failed.org. RbLiK6DPs56eoW1fqt5zF9VxC42UoRGD i3E9fSKtFDgcbjT6IBfbxQw/6trNXuLG J4Zlyrp6KbQ2SgDnw5QWHukTcf9uI/Mo 543mbPI6ooMpejnkr/n0gwntWdfCVoDP Ynej4bA8BaSOSZKVD5QvZPK1/BZezFes TC0aToyZxEc=

 AUTHORITY:

 ADDITIONAL:


In [817]:
d_query = dns.message.make_query('dnssec-failed.org.', 'DNSKEY', want_dnssec=True)
response6 = dns.query.udp(d_query, '68.87.72.244')
print_response(response6)


 QUESTION:
dnssec-failed.org. IN DNSKEY

 ANSWER:
dnssec-failed.org. 3600 IN DNSKEY 256 3 5 AwEAAewq/QcrsNX3C/nAAWyNY74f/q9R b2dGLc3LOIkQBATwzIcDTDHNRjtRDxjq uImNpoDKybI2hZ2e8mNKvCK/F/QXV5La fLwSzscqwvzJxEGZUA+JuiGu6kq/8OjE 6EEAdYlk4ztN6OWfwuqj4ZolBjKPXCPo dYvhj8gl7kqpopqr
dnssec-failed.org. 3600 IN DNSKEY 257 3 5 AwEAAb/f/pB/FLWoYp3j+HtldGkbUMT6 caAw2rej0DZkgXVFOKn4PWi3BYjCozjE qxeramt+9b1SMuOSJ8vGKWr0YKrfyfJi gsVxpsMgJ7QWcxeMACjC/oM8BPjDFBby /CgQQE63nPVX2SfDWCRhEhTOnsPZpKJv q66IHF/w+3u0IpyeplQWvO+HJ9OQPOQr stM7d/IPa7yKEtqS2nhBT0GWX2/GYhT6 oE7F4vc2VF9f6MjpB/pWPzkcx636YaxG 9P0QRBvzdD/Wztcbz1Scgxw5sUlIkQAz WV1mJfvXF+7NqzGcc94/kMt1VUzN2kYA SRyn1ALiFPfNLz4VMUvSw5fpNS0=
dnssec-failed.org. 3600 IN RRSIG DNSKEY 5 2 3600 20180220173528 20180213143028 44973 dnssec-failed.org. JoWy+ptkI9/p1US/QQcXXb157GWRQRGt 3Nbqz2AVxaESdmLlT2P0sHTi6XOF3G6W iooOuuEmdYAH3aknfmR4B2Vi7d4Wr0Tp 299e9TYAtE0U3ZSgsiTUh5byDhtZfJyY qJXphB6aviiafL9HMGkeo9mWXG4FNB21 GyOs8dMofJY=
dnssec-failed.org. 3600 IN RRSIG DNSKEY 5 

In [820]:
def verify_a(response, name_key, dnskey):
    '''Verify the A record in the response.
    
    Args:
        response (dns.message.Message): a response that contains A record
        name_key (dns.name.Name): name of zone that contains the DNSKEY
        dnskey (dns.rrset.RRset): rrset that contains public zone signing key
    '''
    try:
        a, rrsig_a, name_a = get_rrset(response, 'A')
        dns.dnssec.validate(a, rrsig_a, {name_key:dnskey})
    except Exception as e:
        raise e
    else:
        print('Congrats! A records are good~')

In [822]:
name_key, dnskey = verify_dnskey(response6)
verify_a(response5, name_key, dnskey)
verify_zone(response6, response3)

Congrats! DNSKEYs are good~ dnssec-failed.org.
Congrats! A records are good~


Exception: ('DS does not match!', <DNS name dnssec-failed.org.>)